**IMPORTS**

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import os
import csv
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,  precision_score, recall_score
from skimage.feature import graycomatrix, graycoprops
import cv2.ximgproc

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
image = None
texture = None
species = None
fractal_dim = None
entropy = None
vein_density = None
color = None
file_path = '../csv/empty-sheet-columns.csv'

**FUNCTIONS FOR PROCESSING IMAGE**

In [3]:
def show(img_path):
  plt.imshow(img_path)
  plt.axis('off')
  plt.show()

def resizeImage(image):
  try:
    # Get the dimensions of the original image
    original_height, original_width, _ = image.shape

    # Specify the desired dimensions of the resized image
    desired_width = 200
    desired_height = 300

    # Calculate the aspect ratio of the original image
    aspect_ratio = original_width / original_height

    # Apply the aspect ratio to the desired dimensions
    if desired_width / aspect_ratio <= desired_height:
        # Use the desired width to calculate the height
        new_width = desired_width
        new_height = int(new_width / aspect_ratio)
    else:
        # Use the desired height to calculate the width
        new_height = desired_height
        new_width = int(new_height * aspect_ratio)

    # Resize the image
    resized_image = cv2.resize(image, (new_width, new_height))
        # Get dimensions of image
    height, width, channels = resized_image.shape

    # Calculate coordinates of middle pixel
    x = width // 2
    y = height // 2
    global color
    # Get color of middle pixel
    color = resized_image[y, x]
    color  = int(0.299*color[2] + 0.587*color[1] + 0.114*color[0])
    return resized_image
  except Exception as e:
    print("Error in resizing image", e)
    return image
def getDiameter(contour, circularity, rectangularity, ellipticity,area ):
  try:
    # Fit an ellipse to the largest contour
    if ellipticity > circularity and rectangularity:
      ellipse = cv2.fitEllipse(contour)

    # Extract the major and minor axes of the ellipse
      major_axis = max(ellipse[1])
      minor_axis = min(ellipse[1])

    # Calculate the diameter as the maximum of major and minor axes
      diameter = max(major_axis, minor_axis)
      
    elif circularity > ellipticity and rectangularity:
      diameter = 2 * np.sqrt((area * circularity**2) / np.pi)
    else:
      diameter = np.sqrt(area / rectangularity)
    return diameter
  except Exception as e:
    print("Error in getting the diameter", e)
    return 0  
def get_entropy(image):
    """Calculate the entropy of an image."""
    hist, _ = np.histogram(image, bins=np.arange(0, 257))
    hist = hist / np.sum(hist)
    hist = hist[hist > 0]
    return str(-np.sum(hist * np.log2(hist)))
def getTexture(path):
  try:
    global fractal_dim
    global entropy
    # Read in leaf image
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

    # Compute grey-level co-occurrence matrix (GLCM)
    glcm = graycomatrix(img, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], symmetric=True, normed=True)

    # Compute contrast using GLCM
    contrast = graycoprops(glcm, 'contrast')

    # Compute entropy using GLCM
    entropy_val = get_entropy(img)
    

    # Compute fractal dimension using box counting method
    pixels = np.sum(img <= 127)
    scales = np.logspace(0, np.log2(min(img.shape))-1, num=20, base=2, dtype=int)
    ns = []
    for scale in scales:
        slices = [slice(0, s, scale) for s in img.shape]
        ns.append(np.sum(img[tuple(slices)] <= 127))  # fixed line
    coeffs = np.polyfit(np.log(scales), np.log(ns), 1)
    fractal_dim = -coeffs[0]
    fractal_dim = -coeffs[0]
    entropy = entropy_val
  except Exception as e:
    print("Error in getting Texture of the image", e)
    return None
def applyFilters(image):
  global vein_density
  try:
    # Apply Gaussian blur to reduce noise
    ksize = (3, 3)  # kernel size
    sigmaX = 3  # standard deviation in X direction
    blur = cv2.GaussianBlur(image, ksize, sigmaX)
    gray = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    # Apply vein detection algorithm
    dst = cv2.ximgproc.thinning(thresh, thinningType=cv2.ximgproc.THINNING_GUOHALL)

    # Calculate vein density
    num_veins = np.count_nonzero(dst)
    leaf_area = np.count_nonzero(thresh)
    vein_density = num_veins / leaf_area

    # Apply Median blur to reduce noise
    ksize = 5  # kernel size (must be odd)
    median = cv2.medianBlur(blur, ksize)

    # Apply Bilateral filter to reduce noise
    d = 1  # diameter of each pixel neighborhood
    sigmaColor = 100  # filter sigma in the color space
    sigmaSpace = 100  # filter sigma in the coordinate space
    bilateral = cv2.bilateralFilter(median, d, sigmaColor, sigmaSpace)

    return bilateral
  except Exception as e:
    print("Error in applying Filters image", e)
    return image
def flatten(filtered_image):
  try:
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(filtered_image, cv2.COLOR_BGR2GRAY)
    # Perform adaptive histogram equalization to normalize the image
    # You can adjust the clipLimit and tileGridSize parameters to control the normalization effect
    clahe = cv2.createCLAHE(clipLimit=0.5, tileGridSize=(8, 8))
    normalized_image = clahe.apply(gray_image)

    # Convert the normalized image back to BGR color space
    normalized_image = cv2.cvtColor(normalized_image, cv2.COLOR_GRAY2BGR)
    # Resize the mask to match the size of the input image
    mask = cv2.threshold(gray_image, 25, 255, cv2.THRESH_BINARY)[1]
    mask = cv2.resize(mask, (filtered_image.shape[1], filtered_image.shape[0]), interpolation=cv2.INTER_NEAREST)

    # Perform bitwise_and operation on the image and mask
    output_image = cv2.bitwise_and(filtered_image, filtered_image, mask=mask)

    # Perform bitwise_not operation on the mask
    not_mask = cv2.bitwise_not(mask)
    not_mask = cv2.resize(not_mask, (normalized_image.shape[1], normalized_image.shape[0]), interpolation=cv2.INTER_NEAREST)

    normalized_not_mask = cv2.bitwise_and(normalized_image, normalized_image, mask=not_mask)

    # Add the bitwise_and results to get the final output image
    output_image = cv2.add(output_image, normalized_not_mask)

    return output_image
  except Exception as e:
    print("Error in flattening image", e)
    return filtered_image
def remove_background(flattened_image):
  try:
    mask = np.zeros(flattened_image.shape[:2], np.uint8)
    rect = (10, 10, flattened_image.shape[1]-20, flattened_image.shape[0]-20)
    cv2.grabCut(flattened_image, mask, rect, None, None, 5, cv2.GC_INIT_WITH_RECT)

    # Create a binary mask from the grabcut mask
    binary_mask = np.where((mask == 2) | (mask == 0), 0, 1).astype(np.uint8)

    # Apply the binary mask to the original image
    flattened_image = cv2.bitwise_and(flattened_image, flattened_image, mask=binary_mask)


    return flattened_image
  except Exception as e:
    print("Error in removing background", e)
    return flattened_image

def get_edge(image):
  try:
    # Apply Scharr operator for x and y directions
    scharr_x = cv2.Scharr(image, cv2.CV_64F, 1, 0)
    scharr_y = cv2.Scharr(image, cv2.CV_64F, 0, 1)

    # Compute the gradient magnitude and direction
    grad_mag = cv2.magnitude(scharr_x, scharr_y)
    grad_dir = cv2.phase(scharr_x, scharr_y, angleInDegrees=True)

    threshold_value = 200  # Adjust this threshold value as needed
    _, edges = cv2.threshold(grad_mag, threshold_value, 255, cv2.THRESH_BINARY)


    edges = edges.astype(np.uint8)
    return edges
  except Exception as e:
    print("Error in getting edge", e)
    return image

def get_contours(edges,image):
  try:
    edge = cv2.Canny(edges, 50, 100)


    # Perform dilation on the edges
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    dilated_edges = cv2.dilate(edge, kernel, iterations=1)

    # Perform erosion on the edges
    eroded_edges = cv2.erode(edge, kernel, iterations=3)
    # Find contours in the binary image
    contours, hierarchy = cv2.findContours(dilated_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnt =max(contours, key=cv2.contourArea)
    return cnt
  except Exception as e:
    print("Error in getting contours", e)
    return None
def get_features(cnt):
  try:
    moments = cv2.moments(cnt)

    # Calculate circularity
    perimeter = cv2.arcLength(cnt, True)
    area = cv2.contourArea(cnt)
    circularity = 4 * np.pi * area / (perimeter ** 2)
        
    # Calculate rectangularity
    x,y,w,h = cv2.boundingRect(cnt)
    rectangularity = w/h
        
    # Calculate ellipticity
    a = moments['mu20'] + moments['mu02']
    b = np.sqrt(4 * moments['mu11'] ** 2 + (moments['mu20'] - moments['mu02']) ** 2)
    ellipticity = np.sqrt((a + b) / (a - b))

    x, y, w, h = cv2.boundingRect(cnt)
    length = max(w, h)
    width = min(w, h)
    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt, True)
    diameter = getDiameter(cnt, circularity, rectangularity, ellipticity,area)

    apect_ratio = length/width
    form_factor = (4*math.pi*area)/perimeter**2
    narrow_factor = diameter/length
    perimeter_ratio_of_diameter = perimeter/diameter
    perimeter_ratio_of_PLW = perimeter/(length+width)

    # Calculate Eccentricity
    ellipse = cv2.fitEllipse(cnt)
    eccentricity = 1 - (ellipse[1][0] / ellipse[1][1])

    # Calculate Curvature
    #epsilon = 0.01 * cv2.arcLength(cnt, True)
    #approx = cv2.approxPolyDP(cnt, epsilon, True)
    #curvature = len(approx) / cv2.arcLength(cnt, True)
    x, y = cnt[:, 0, 0], cnt[:, 0, 1]
    x_prime, y_prime = np.gradient(x), np.gradient(y)
    x_double_prime, y_double_prime = np.gradient(x_prime), np.gradient(y_prime)
    curvature = (x_prime * y_double_prime - y_prime * x_double_prime) / ((x_prime ** 2 + y_prime ** 2) ** 1.5)

    # Count teeth
    teeth = 0
    for i in range(1, len(curvature)):
        if curvature[i] * curvature[i-1] < 0:
            teeth += 1
    curvature = np.abs(curvature).sum() / len(curvature)
    features =[length,width,diameter,area,perimeter,rectangularity,circularity,ellipticity,
                        apect_ratio,form_factor,narrow_factor,perimeter_ratio_of_diameter,perimeter_ratio_of_PLW,fractal_dim,
                        entropy, eccentricity,curvature,vein_density, color,teeth, species]

    return features

  except Exception as e:
    print("Error in getting features", e)
    return None

def process_image(image):
  # Main
  # Load an image
  try:
    image=resizeImage(image)
    filtered_image = applyFilters(image)
    flattened_image = flatten(filtered_image)
    background_removed = remove_background(flattened_image)
    edge = get_edge(background_removed)
    cnt = get_contours(edge,image)
    features = get_features(cnt)
    return features
  except Exception as e:
    print("Error")

FUNCTIONS FOR INPUTTING DATA TO TABLE


In [14]:

import csv

def read_csv_features(filename):
    with open(filename, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        header = next(csvreader)
        data = {col: [] for col in header}
        for row in csvreader:
            for col, val in zip(header, row):
                data[col].append(val)
    return data


def append_data_to_csv(file_path, data):
    try:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Create a new DataFrame with the data to be appended
        new_data = pd.DataFrame([data], columns=df.columns)
        
        # Append the new data to the original DataFrame
        df = pd.concat([df, new_data], ignore_index=True)
        
        # Write the updated data back to the CSV file
        df.to_csv(file_path, index=False)
        
        print("Data has been appended to the CSV file successfully!")
    except FileNotFoundError:
        print("File not found. Please check the file path.")
    except Exception as e:
        print("Error occurred:", str(e))


def read_csv_to_table(file_path):
    try:
        # Read CSV file into a pandas DataFrame
        df = pd.read_csv(file_path)      
        return df

    except FileNotFoundError:
        print("File not found. Please check the file path.")
    except Exception as e:
        print("Error occurred:", str(e))

def loop_through_directory(dir_path):
    # Loop through all the contents of the directory
  for item in os.listdir(dir_path):
      # Create the absolute path by joining the directory path and the item name
      item_path = os.path.join(dir_path, item)
      
      # Check if the item is a file
      if os.path.isfile(item_path):
        image_path = item_path
        
        # Open the image
        image = cv2.imread(image_path)
        getTexture(image_path)
        data = np.array(process_image(image))

        try:
            for i in range(len(data)):
                key = keys_list[i % len(keys_list)]
                data_dict[key].append(data[i])
        except Exception:
            pass
    
        #print(len(data_dict['length']))

        #append data to csv file
        #append_data_to_csv(file_path,data)

In [15]:
import os

folder_path = "../images/compilation"
folder_paths = []

for root, dirs, files in os.walk(folder_path):
    for dir in dirs:
        folder_paths.append(os.path.join(root, dir))

print(folder_paths)
print(len(folder_paths))

['../images/compilation\\Blumea_balsamifera', '../images/compilation\\Ehretia_microphylla', '../images/compilation\\Psidium_guajava', '../images/compilation\\salix_babylonica', '../images/compilation\\salix_caroliniana', '../images/compilation\\salix_matsudana', '../images/compilation\\salix_nigra', '../images/compilation\\styrax_japonica', '../images/compilation\\styrax_obassia', '../images/compilation\\tilia_americana', '../images/compilation\\tilia_cordata', '../images/compilation\\tilia_europaea', '../images/compilation\\tilia_tomentosa', '../images/compilation\\ulmus_americana', '../images/compilation\\ulmus_glabra', '../images/compilation\\ulmus_parvifolia', '../images/compilation\\ulmus_pumila', '../images/compilation\\ulmus_rubra', '../images/compilation\\Vitex_nagundo']
19


In [16]:
plant_names = [string.split('\\')[1] for string in folder_paths]
plant_names

['Blumea_balsamifera',
 'Ehretia_microphylla',
 'Psidium_guajava',
 'salix_babylonica',
 'salix_caroliniana',
 'salix_matsudana',
 'salix_nigra',
 'styrax_japonica',
 'styrax_obassia',
 'tilia_americana',
 'tilia_cordata',
 'tilia_europaea',
 'tilia_tomentosa',
 'ulmus_americana',
 'ulmus_glabra',
 'ulmus_parvifolia',
 'ulmus_pumila',
 'ulmus_rubra',
 'Vitex_nagundo']

**FUNCTION FOR MAIN**

In [17]:
data = read_csv_features(file_path)
global keys_list 
keys_list = list(data.keys())

keys_list

['length',
 'width',
 'diameter',
 'area',
 'perimeter',
 'rectangularity',
 'circularity',
 'ellipticity',
 'apect_ratio',
 'form_factor',
 'narrow_factor',
 'perimeter_ratio_of_diameter',
 'perimeter_ratio_of_PLW',
 'fractal_dim',
 'entropy',
 'eccentricity',
 'curvature',
 'vein_density',
 'color',
 'teeth',
 'species']

In [18]:
# create dictionary with named keys and empty lists
global data_dict 
data_dict = {key: [] for key in keys_list}

for i,folder_path in enumerate(folder_paths):
    dir_path = folder_path
    global species
    species = i
    print(species)
    loop_through_directory(dir_path)
    
    #read_csv_to_table(file_path)


0
Error in getting contours max() arg is an empty sequence
Error in getting features OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\shapedescr.cpp:280: error: (-215:Assertion failed) count >= 0 && (depth == CV_32F || depth == CV_32S) in function 'cv::arcLength'

Error in getting contours max() arg is an empty sequence
Error in getting features OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\shapedescr.cpp:280: error: (-215:Assertion failed) count >= 0 && (depth == CV_32F || depth == CV_32S) in function 'cv::arcLength'

Error in getting contours max() arg is an empty sequence
Error in getting features OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\shapedescr.cpp:280: error: (-215:Assertion failed) count >= 0 && (depth == CV_32F || depth == CV_32S) in function 'cv::arcLength'

Error in getting contours max() arg is an empty sequence
Error in getting features OpenCV(4.7.0) D:\a\opencv-python\opencv-python\open

C:\Users\Jeremy\AppData\Local\Temp\ipykernel_14000\3669796130.py:95: RuntimeWarning: divide by zero encountered in log
  coeffs = np.polyfit(np.log(scales), np.log(ns), 1)


5
6
7
8
9
10
11
12
13
14
15
16
17
18
Error in getting contours max() arg is an empty sequence
Error in getting features OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\shapedescr.cpp:280: error: (-215:Assertion failed) count >= 0 && (depth == CV_32F || depth == CV_32S) in function 'cv::arcLength'

Error in getting contours max() arg is an empty sequence
Error in getting features OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\shapedescr.cpp:280: error: (-215:Assertion failed) count >= 0 && (depth == CV_32F || depth == CV_32S) in function 'cv::arcLength'

Error in getting contours max() arg is an empty sequence
Error in getting features OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\shapedescr.cpp:280: error: (-215:Assertion failed) count >= 0 && (depth == CV_32F || depth == CV_32S) in function 'cv::arcLength'

Error in getting contours max() arg is an empty sequence
Error in getting features OpenCV(4.7.0) D:

In [19]:
df = pd.DataFrame.from_dict(data_dict)
df

,length,width,diameter,area,perimeter,rectangularity,circularity,ellipticity,apect_ratio,form_factor,...,perimeter_ratio_of_diameter,perimeter_ratio_of_PLW,fractal_dim,entropy,eccentricity,curvature,vein_density,color,teeth,species
0,177,146,180.91751098632812,16650.0,539.6122617721558,1.2123287671232876,0.7185557721734613,1.5825439061326507,1.2123287671232876,0.7185557721734613,...,2.9826425249291324,1.6706261974370147,1.8909628944777308,4.896521204508982,0.3551550569682228,0.1584203839497982,0.027938291894184383,151,58,0
1,168,97,173.08502197265625,13001.0,467.3624794483185,1.731958762886598,0.7479609680187818,1.7276789966298514,1.731958762886598,0.7479609680187818,...,2.7001901962502095,1.763631997918183,1.8455973560562149,4.855767315990042,0.4140427829643931,0.22484337636292193,0.029066213921901528,231,37,0
2,181,66,164.96405029296875,7281.5,451.60511922836304,2.742424242424242,0.448655509541637,2.9781303427274852,2.742424242424242,0.448655509541637,...,2.7375971820910836,1.828360806592563,1.9446637486438654,4.720796862905966,0.6400815820134225,0.15290892802144884,0.038161625708884685,141,43,0
3,187,152,189.2084503173828,17753.0,572.3574262857437,1.230263157894737,0.6809997819388067,1.5652889039602187,1.230263157894737,0.6809997819388067,...,3.025009851968332,1.6883699890434918,1.852114271558875,4.0621038075130915,0.3436003246783358,0.14328925315323024,0.020291693088142042,215,47,0
4,287,133,289.0489807128906,30347.0,766.9015842676163,0.4634146341463415,0.6484050055068105,2.178898942880523,2.1578947368421053,0.6484050055068105,...,2.6531890282961132,1.825956153018134,1.8201552193306352,4.927389978216777,0.5164271301962285,0.41460081056039805,0.019083073414495087,195,48,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34092,165,83,174.65187072753906,5400.0,408.13203060626984,1.9879518072289157,0.4073824014113313,4.333301458122077,1.9879518072289157,0.4073824014113313,...,2.3368317150347915,1.6456936717994752,1.9404717684467359,0.7926769377213666,0.7657769296395198,0.05529543852621024,0.04355203619909502,193,35,18
34093,276,107,281.5074157714844,18000.5,680.457932472229,0.38768115942028986,0.48853071075753896,3.204195414513536,2.5794392523364484,0.48853071075753896,...,2.4171936309649316,1.7766525652016423,1.9215044323498234,3.7809886623266946,0.6894494622163851,0.09469702506881268,0.02229511842544201,103,56,18
34094,155,47,153.9854736328125,5246.5,370.6934322118759,3.297872340425532,0.47978841616697404,3.5893575011668557,3.297872340425532,0.47978841616697404,...,2.407327285272482,1.8351160010488907,1.9550554928920192,3.962493260547061,0.7138396994250262,0.13589459399463086,0.04506946797695696,118,30,18
34095,287,97,263.73883056640625,17571.5,720.6589410305023,0.33797909407665505,0.4251666023264165,3.4028046313824563,2.9587628865979383,0.4251666023264165,...,2.732471890782306,1.8767159922669332,1.9104068066512114,4.519591148255138,0.6119482795688225,0.2007013254211697,0.02359452374016895,89,49,18


In [21]:
df.to_csv("../csv/fin_features.csv")

In [ ]:
print(cv2.__version__)

4.7.0


In [ ]:
def main():
    # Specify the directory path
    dir_path = 'images'
    global species
    species = 1
    loop_through_directory(dir_path)
    read_csv_to_table(file_path)


In [ ]:
main()